In [1]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Input, concatenate
from keras.utils import np_utils
from keras.optimizers import SGD, Adagrad, RMSprop, Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Lambda
import glob
import cv2
import itertools
from skimage.transform import resize
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

Using TensorFlow backend.


In [2]:
batch_size = 2000
gt_all = os.listdir('gt')

X = np.zeros((batch_size,256,256,3))
y = np.zeros((batch_size,256,256,3))
for a in range(batch_size):
    i = random.randint(0,len(gt_all)-1)

    name = gt_all[i]
    name = name.replace('_segmentation','')
    name = name.replace('png','jpg')
    img = cv2.imread(os.path.join('gt',gt_all[i]))
    thresh = 127
    img = cv2.resize(img,(256,256))
    img = cv2.threshold(img, thresh, 1, cv2.THRESH_BINARY)[1]
    y[a] = img
    if name in os.listdir('melanoma'):
        img = cv2.imread(os.path.join('melanoma',name))
        if img is not None:
            img = cv2.resize(img,(256,256))
            X[a] = img
    else:
        img = cv2.imread(os.path.join('others',name))
        if img is not None:
            img = cv2.resize(img,(256,256))
            X[a] = img

In [6]:
smooth = 1.

def dice_coef(y_true, y_pred):
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    
    return -dice_coef(y_true, y_pred)

In [45]:
nb_epoch = 10
batch_size = 10

In [72]:
inputs = Input((256, 256, 3))

conv1 = Conv2D(8, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(8, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)


model = Model(inputs=[inputs], outputs=[conv10])

In [73]:
model.compile(loss = dice_coef_loss, optimizer=Adam(lr=1e-5), metrics=[dice_coef, 'accuracy'])

model.fit(segment_train_X, segment_train_Y, batch_size=batch_size, epochs = nb_epoch, validation_split=0.2, shuffle=True)

Train on 1280 samples, validate on 320 samples
Epoch 1/10
1280/1280 [==============================] - 120s 93ms/step - loss: -0.2456 - dice_coef: 0.2456 - acc: 0.7978 - val_loss: -0.4088 - val_dice_coef: 0.4088 - val_acc: 0.7975
Epoch 2/10
1280/1280 [==============================] - 109s 85ms/step - loss: -0.4860 - dice_coef: 0.4860 - acc: 0.8288 - val_loss: -0.5407 - val_dice_coef: 0.5407 - val_acc: 0.8531
Epoch 3/10
1280/1280 [==============================] - 106s 83ms/step - loss: -0.5785 - dice_coef: 0.5785 - acc: 0.8573 - val_loss: -0.6025 - val_dice_coef: 0.6025 - val_acc: 0.8598
Epoch 4/10
1280/1280 [==============================] - 106s 83ms/step - loss: -0.6317 - dice_coef: 0.6317 - acc: 0.8718 - val_loss: -0.6334 - val_dice_coef: 0.6334 - val_acc: 0.8637
Epoch 5/10
1280/1280 [==============================] - 111s 86ms/step - loss: -0.6574 - dice_coef: 0.6574 - acc: 0.8802 - val_loss: -0.6710 - val_dice_coef: 0.6710 - val_acc: 0.8914
Epoch 6/10
1280/1280 [================

In [74]:
score = model.evaluate(segment_test_X, segment_test_Y)

400/400 [==============================] - 11s 28ms/step


In [76]:
print(score[2] * 100)

90.6045227051
